In [0]:
filepath = "/Volumes/workspace/default/test_data/bronze/transactions/transaction.snappy.parquet"
df = spark.read.parquet(filepath)
display(df)

In [0]:
df.printSchema()

In [0]:
from pyspark.sql.functions import col,to_timestamp
df_new = df.withColumn("transaction_date", to_timestamp(col("transaction_date")))
df_new.printSchema()

In [0]:
display(df_new)

In [0]:
from pyspark.sql.functions import current_timestamp
final_df = df_new.withColumn("ingestion_timestamp", current_timestamp())
display(final_df)

In [0]:
spark.sql("use globalretail_bronze")
final_df.write.format("delta").mode("append").saveAsTable("bronze_transactions")
spark.sql("select * from bronze_transactions limit 10").show()

In [0]:
%sql
select table_name 
from workspace.information_schema.tables 
where table_schema = 'globalretail_bronze'

In [0]:
import datetime
import shutil
import os

archive_folder = "/Volumes/workspace/default/test_data/bronze/transactions/archive/"
archive_filepath = archive_folder + "_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".parquet"

# Ensure the archive directory exists
os.makedirs(archive_folder, exist_ok=True)

shutil.move(filepath, archive_filepath)
print(archive_filepath)